In [30]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(temperature=0.1)

cache_dir = LocalFileStore("./.cache/")
memory = ConversationBufferMemory(return_messages=True)

splitter = CharacterTextSplitter.from_tiktoken_encoder(separator="\n", chunk_size=600, chunk_overlap=100)
loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever()

map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm

def map_docs(inputs):
    documents = inputs["documents"]
    question = str(inputs["question"])
    responses = [
        map_doc_chain.invoke({"context": doc.page_content, "question": question}).content
        for doc in documents
    ]
    return "\n\n".join(filter(None, responses)) if responses else "No relevant information found."

map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough() | RunnableLambda(lambda x: str(x)),
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {history}
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    history = memory.load_memory_variables({})["history"]
    print(history)
    return "\n".join(
        [f"{msg['role']}: {msg['content']}" for msg in history if 'role' in msg and 'content' in msg]
    ) if history else ""


chain = RunnablePassthrough() | {
    "history": RunnableLambda(load_memory),
    "context": map_chain,
    "question": RunnablePassthrough(),
} | final_prompt | llm

def invoke_chain(question):
    question = str(question) 
    result = chain.invoke({"question": question})
    output_content = str(result.content) if result.content else "No response generated."
    memory.save_context({"input": question}, {"output": output_content})
    print(f"🔍 Question: {question}\n💬 AI Response: {output_content}\n")

questions = [
    "Is Aaronson guilty?",
    "What message did he write on the table?",
    "Who is Julia?"
]

for question in questions:
    invoke_chain(question)
    print('-' * 50)

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


KeyError: "Input to ChatPromptTemplate is missing variables {'history'}.  Expected: ['context', 'history', 'question'] Received: ['context', 'question']\nNote: if you intended {history} to be part of the string and not a variable, please escape it with double curly braces like: '{{history}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "